Reading for HW5
--------------------------------------------------------------------------------
This time, to understand the task and network we are using, you need to
understand some papers from Google research and our nice colleges:

Understand counting: https://www3.cs.stonybrook.edu/~minhhoai/papers/fewshot_counting_CVPR21.pdf

understand attention: https://www.youtube.com/watch?v=ptuGllU5SQQ
Stanford CS224N NLP with Deep Learning | Winter 2021 | Lecture 9 - Self- Attention and Transformers

understand ViT: https://github.com/google-research/vision_transformer
[An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

few-shot counting using Vision Transformer
--------------------------------------------------------------------------------

In this programing assignment, we would train a Network to
count objects in a image.
Find the date set here:
https://drive.google.com/file/d/1QVr5_wRTiRiCDKRVaN_bdb3Rm_IJaD0I/view?usp=sharing
Find the pretrained model parameter here:
https://drive.google.com/file/d/1PujXdKtUAVdrCaEeeTorFO5G7trWvs8S/view?usp=sharing


Put the compressed file inside the data folder, uncompress it.
You would see fellowing structure
- gt_density_map_adaptive_384_VarV2
- json_annotationCombined_384_VarV2.json
- Train_Test_Val_FSC_147.json
- images_384_VarV2

two folders contains 6,146 images and labels
two json file for data precessing, you wouldn't need to modify data prepcessing part

This is an experimental model. Most part of model are completed expect
attention part and a fully connected layer part. you would need to implement
this module exactly as described in order to load pre-trained matrix

Note0: refer to doc/More_Details.pdf for details   \\
Note1: need to copy your code into lib/VIT.py, it's identical structure \\
Note2: it has a long training time, let it run over night and see what's the best
        loss value you can get
HINT:
```shell
    (11): EncoderBlock(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): SelfAttention(
        (query): LinearGeneral()
        (key): LinearGeneral()
        (value): LinearGeneral()
        (out): LinearGeneral()
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MlpBlock(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (act): GELU()
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
```

![img](./doc/model.JPG)

In [ ]:
drive.flush_and_unmount()

In [ ]:
# I've executed the entire notebook in another account and then put the notebook here, since my collab limits had reached for both my 
# stonybrook and cs.stonybrook account. You can find the log and weight files for both the train runs(normal and stuck in local minima)
# in the shared project folder. 

In [ ]:

import os
from google.colab import drive
drive._mount('/content/gdrive',force_remount=True)




Mounted at /content/gdrive


# Load Data(Unzip everytime can increase the speed)

In [ ]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/CV_2021Fall_HW5/data')
!ls
!unzip CV_HW5_data.zip

os.chdir('/content/gdrive/My Drive/Colab Notebooks/CV_2021Fall_HW5/')
!/opt/bin/nvidia-smi
!ls

Streaming output truncated to the last 5000 lines.
  inflating: images_384_VarV2/3869.jpg  
  inflating: __MACOSX/images_384_VarV2/._3869.jpg  
  inflating: images_384_VarV2/526.jpg  
  inflating: __MACOSX/images_384_VarV2/._526.jpg  
  inflating: images_384_VarV2/6905.jpg  
  inflating: __MACOSX/images_384_VarV2/._6905.jpg  
  inflating: images_384_VarV2/240.jpg  
  inflating: __MACOSX/images_384_VarV2/._240.jpg  
  inflating: images_384_VarV2/4874.jpg  
  inflating: __MACOSX/images_384_VarV2/._4874.jpg  
  inflating: images_384_VarV2/1924.jpg  
  inflating: __MACOSX/images_384_VarV2/._1924.jpg  
  inflating: images_384_VarV2/6093.jpg  
  inflating: __MACOSX/images_384_VarV2/._6093.jpg  
  inflating: images_384_VarV2/268.jpg  
  inflating: __MACOSX/images_384_VarV2/._268.jpg  
  inflating: images_384_VarV2/3855.jpg  
  inflating: __MACOSX/images_384_VarV2/._3855.jpg  
  inflating: images_384_VarV2/6939.jpg  
  inflating: __MACOSX/images_384_VarV2/._6939.jpg  
  inflating: images_384_V

In [ ]:

"""
# Counting Transformer Model
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Position Embedding in the VIT
class PositionEmbs(nn.Module):
    def __init__(self, Max_num_patches, emb_dim, dropout_rate=0.1):
        super(PositionEmbs, self).__init__()
        self.pos_embedding = nn.Parameter(torch.randn(1, Max_num_patches, emb_dim))
        if dropout_rate > 0:
            self.dropout = nn.Dropout(dropout_rate)
        else:
            self.dropout = None

    def forward(self, x):
        patches = x.shape[0]
        pos_embedding = self.pos_embedding.squeeze()
        out = x + pos_embedding[0:patches]

        if self.dropout:
            out = self.dropout(out)
        out = out.unsqueeze(0)
        return out


class MlpBlock(nn.Module):
    """ Transformer Feed-Forward Block """
    def __init__(self, in_dim, mlp_dim, out_dim, dropout_rate=0.1):
        super(MlpBlock, self).__init__()
        """
            PART I: 20 points
            FC layer(with dropout) + GELU act layer + FC layer(with dropout)
        """
        """ STUDENT CODE START """
        self.fc1 = nn.Linear(in_dim,mlp_dim)
        # torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.drp1 = nn.Dropout(dropout_rate)
        
        self.gelu = nn.GELU()

        self.fc2 = nn.Linear(mlp_dim,out_dim)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        self.drp2 = nn.Dropout(dropout_rate)
        """ STUDENT CODE END """

    def forward(self, x):

        """ STUDENT CODE START """
        out = self.fc1(x)
        out = self.drp1(out)
        out = self.gelu(out)
        out = self.fc2(out)
        out = self.drp2(out)
        """ STUDENT CODE END """
        return out

# For implement the self-attention
class LinearGeneral(nn.Module):
    def __init__(self, in_dim=(768,), feat_dim=(12, 64)):
        super(LinearGeneral, self).__init__()

        self.weight = nn.Parameter(torch.randn(*in_dim, *feat_dim))
        self.bias = nn.Parameter(torch.zeros(*feat_dim))

    def forward(self, x, dims):
        a = torch.tensordot(x, self.weight, dims=dims) + self.bias
        return a

# Multi-head attention
class SelfAttention(nn.Module):
    def __init__(self, in_dim, heads=8, dropout_rate=0.1):
        super(SelfAttention, self).__init__()

        """
            PART II: 40 points
            multihead SelfAttention part with Key, Query, Value
            use LinearGeneral class
        """
        """ STUDENT CODE BEGIN """
        self.heads = heads
        self.grps = in_dim // heads
        self.rootDk = self.grps ** 0.5

        self.query = LinearGeneral((in_dim,),(self.heads,self.grps))
        self.key = LinearGeneral((in_dim,),(self.heads,self.grps))
        self.value = LinearGeneral((in_dim,),(self.heads,self.grps))
        
        self.out = LinearGeneral((self.heads,self.grps),(in_dim,))


        """ STUDENT CODE END"""

    def forward(self, x):

        """ STUDENT CODE BEGIN """
        # print("In self attention")
        # print(x.shape)
        batch, n, _ = x.shape

        q = self.query(x,dims=([2],[0]))
        k = self.key(x,dims=([2],[0]))
        v = self.value(x,dims=([2],[0]))
        q = q.swapaxes(1,2)
        k = k.swapaxes(1,2)
        v = v.swapaxes(1,2)

        alpha = torch.matmul(q, k.transpose(-2, -1))
        alpha/= self.rootDk
        alpha = F.softmax(alpha, dim=-1)
        out = torch.matmul(alpha, v)
        out = out.swapaxes(1,2)
        out = self.out(out, dims=([2, 3], [0, 1]))
        """ STUDENT CODE END """
        return out

# Encoder LayerNorm -> Self-attention -> Layernorm -> MLP
class EncoderBlock(nn.Module):
    def __init__(self, in_dim, mlp_dim, num_heads, dropout_rate=0.1, attn_dropout_rate=0.1):
        super(EncoderBlock, self).__init__()

        self.norm1 = nn.LayerNorm(in_dim)
        self.attn = SelfAttention(in_dim, heads=num_heads, dropout_rate=attn_dropout_rate)
        if dropout_rate > 0:
            self.dropout = nn.Dropout(dropout_rate)
        else:
            self.dropout = None
        self.norm2 = nn.LayerNorm(in_dim)
        self.mlp = MlpBlock(in_dim, mlp_dim, in_dim, dropout_rate)

    def forward(self, x):
        residual = x
        out = self.norm1(x)
        out = self.attn(out)
        if self.dropout:
            out = self.dropout(out)
        out += residual
        residual = out

        out = self.norm2(out)
        out = self.mlp(out)
        out += residual
        return out

# 12 Transformer layers
class Encoder(nn.Module):
    def __init__(self, emb_dim, mlp_dim, num_layers=12, num_heads=12, dropout_rate=0.1, attn_dropout_rate=0.0,  Max_num_patches=512):
        super(Encoder, self).__init__()

        # positional embedding
        self.pos_embedding = PositionEmbs(Max_num_patches, emb_dim, dropout_rate)

        # encoder blocks
        in_dim = emb_dim
        self.encoder_layers = nn.ModuleList()
        """
            PART III: 20 points
            Apply 12 layer of transformer and one LyaerNorm
        """
        """ STUDENT CODE START """

        for i in range(num_layers):
            enc_layer = EncoderBlock(in_dim, mlp_dim, num_heads, dropout_rate, attn_dropout_rate)
            self.encoder_layers.append(enc_layer)

        self.norm = nn.LayerNorm(in_dim)

        """ STUDENT CODE END """


    def forward(self, x):

        out = self.pos_embedding(x)

        """ STUDENT CODE START """
        for mid_layer in self.encoder_layers:
            out = mid_layer(out)

        out = self.norm(out)
        """ STUDENT CODE END """
        return out

# Upsampling and conv
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.Conv2d(768, 96, 7, padding=3),
            nn.ReLU(),

            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(96, 64, 5, padding=2),
            nn.ReLU(),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(32, 1, 1),
            nn.ReLU(),
            nn.UpsamplingBilinear2d(scale_factor=2),
        )
    # 4 * 4 * 768 -> 8 * 8 * 96 ->
    def forward(self, input, H, W, B):
        input = input.reshape(B, -1, H, W)
        out = self.decoder(input)
        return out

class CountingTransformer(nn.Module):
    def __init__(self,
                 patch_size=(16, 16),
                 emb_dim=768,
                 mlp_dim=3072,
                 num_heads=12,
                 num_layers=12,
                 attn_dropout_rate=0.0,
                 dropout_rate=0.1,
                 feat_dim=None,
                 Max_num_patches=2000):
        super(CountingTransformer, self).__init__()
        # embedding layer
        self.patch_size = patch_size
        fh, fw = self.patch_size

        # Image_patch -> token
        self.embedding = nn.Conv2d(3, emb_dim, kernel_size=(fh, fw), stride=(fh, fw))

        # image_patch -> token

        # Encoder
        self.transformer = Encoder(
            emb_dim=emb_dim,
            mlp_dim=mlp_dim,
            num_layers=num_layers,
            num_heads=num_heads,
            dropout_rate=dropout_rate,
            attn_dropout_rate=attn_dropout_rate,
            Max_num_patches=Max_num_patches)

        # Decoder
        self.decoder = Decoder()

    # Image and positive patches
    def forward(self, x, positive_token):

        # x image
        # pos exmplar

        # Image shape and patch size
        imh = x.shape[2]
        imw = x.shape[3]
        fh, fw = self.patch_size

        # Patch num
        patch_num = int((imh / fh) * (imw / fw))

        # H, W is the size to reshape at the decoder
        H = int(x.shape[2] / fh)
        W = int(x.shape[3] / fw)
        B = x.shape[0]

        # (H * W) * token_dim -> H * W * C(token_dim)
        # 32 * 600 -> 8 * 4 * 600

        # Embed a image patch into a token
        emb = self.embedding(x)     # (n, c, gh, gw)
        # Embed positive exemplar
        pos_emb = self.embedding(positive_token)

        # Change the shape
        emb = emb.permute(0, 2, 3, 1)  # (n, gh, hw, c)
        b, h, w, c = emb.shape
        emb = emb.reshape(b, h * w, c)
        emb = emb.squeeze()
        pos_emb = pos_emb.squeeze()
        emb = torch.cat((emb, pos_emb))

        # Encoder with image patches and pos exemplar
        feat = self.transformer(emb)

        # Drop the exemplar embedding
        feat_throw_pos = feat[0,0:patch_num,:]

        # Density map
        density_map = self.decoder(feat_throw_pos, H, W, B)

        return density_map


#Check the pipeline
model = CountingTransformer(num_layers=12)
x = torch.randn((1, 3, 384, 256))
token = torch.randn((3, 3, 16, 16))
out = model(x, token)
print(out.shape)


torch.Size([1, 1, 384, 256])


# Training
## Dataloader and Model

In [ ]:

import cv2
import json
import copy
import torch
import datetime
import numpy as np
import torch.optim as optim
import torch.nn.functional as F

from lib.dataset import FscBgDataset

os.chdir('/content/gdrive/My Drive/Colab Notebooks/CV_2021Fall_HW5/')
DATA_DIR = '/content/gdrive/My Drive/Colab Notebooks/CV_2021Fall_HW5/data/'
BATCH_SIZE = 16
PATCH_SIZE = (16,16)
Max_num_patches = 2500
# LR = 5e-6
LR = 1e-5
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = 'cpu'

In [ ]:

from torch.utils.data import DataLoader
train_dataset = FscBgDataset(DATA_DIR, 'train')
val_dataset = FscBgDataset(DATA_DIR, 'val')
test_dataset = FscBgDataset(DATA_DIR, 'test')
im_idx = np.arange(len(train_dataset))
train_loader = DataLoader(im_idx, shuffle=True, batch_size=BATCH_SIZE)
im_idx = np.arange(len(val_dataset))
val_loader = DataLoader(im_idx, shuffle=True, batch_size=BATCH_SIZE)
im_idx = np.arange(len(test_dataset))
test_loader = DataLoader(im_idx, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
!ls
from lib.VIT import VisionTransformer
pretrain_path = './model_save/imagenet21k+imagenet2012_ViT-B_16.pth'
pretrain_model = VisionTransformer(
          image_size=(384, 384),
          patch_size=(16, 16),
          emb_dim=768,
          mlp_dim=3072,
          num_heads=12,
          num_layers=12,
          num_classes=1000,
          attn_dropout_rate=0,
          dropout_rate=0.1)

#Load pre-trained VIT backbone
sdict = torch.load(pretrain_path)
# pretrain_model.load_state_dict(sdict['state_dict'])
# pretrain_model.transformer.encoder_layers = pretrain_model.transformer.encoder_layers[0:3]
# print(pretrain_model)
#Counting Transformer
model = CountingTransformer(num_layers=4,num_heads=5, Max_num_patches=Max_num_patches)
# print(pretrain_model.transformer.encoder_layers.state_dict().shape)
# model.transformer.encoder_layers.load_state_dict(pretrain_model.transformer.encoder_layers.state_dict())
model.to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)

 CV_2021Fall_HW5.public.ipynb			  Log.txt
 data						  model_save
 final_copy_skamat_CV_2021Fall_HW5_public.ipynb  'Run 1 - local optima'
 lib


In [ ]:
import torch.optim as optim
LR = 5e-6
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
optimizer = optim.Adam(model.parameters(), lr=LR)

## Train and test

In [ ]:
from lib.utils import TransformTrain

def train():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in train_loader:
      optimizer.zero_grad()
      if debug:
          if cnt > 20:
            break
      for idx in idx_batch:
        cnt += 1
        train_sample = train_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = train_sample['im_id'], train_sample['image'], train_sample['boxes'], train_sample['dots'], train_sample['bg_mask_img'], train_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        # Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)

          # reshape interpolate
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        # Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        # Feed to the Network
        # Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')

        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)

        #
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Loss 1
        loss = criterion(out_density, GT_density) + 1e-3 * (out_density.sum() - gt_cnt) ** 2

        #Loss 2
        #loss = criterion(out_density, GT_density)

        train_loss += loss.item()

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        loss.backward()
        endtime = datetime.datetime.now()
      optimizer.step()
    print('TRAIN MAE: {:6.2f}, TRAIN RMSE: {:6.2f}, Train Loss: {}, Running Time:{}'.format(SAE/cnt, (SSE/cnt)**0.5, train_loss/cnt, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    TRAIN_LOSS = train_loss/cnt
    return MAE, RMSE, TRAIN_LOSS

def val():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in val_loader:
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        val_sample = val_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = val_sample['im_id'], val_sample['image'], val_sample['boxes'], val_sample['dots'], val_sample['bg_mask_img'], val_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        #Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        #Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        #Feed to the Network
        #Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')
        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        endtime = datetime.datetime.now()
    print('VAL MAE: {:6.2f}, VAL RMSE: {:6.2f}, Running Time: {}'.format(SAE/cnt, (SSE/cnt)**0.5, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    return MAE, RMSE

def test():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in test_loader:
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        test_sample = test_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = test_sample['im_id'], test_sample['image'], test_sample['boxes'], test_sample['dots'], test_sample['bg_mask_img'], test_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        #Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        #Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        #Feed to the Network
        #Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')
        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        endtime = datetime.datetime.now()
    print('TEST MAE: {:6.2f}, TEST RMSE: {:6.2f}, Running Time: {}'.format(SAE/cnt, (SSE/cnt)**0.5, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    return MAE, RMSE


EPOCH = 20
debug = False
stats = list()
Log_Save_dir = './'
best_mae, best_rmse = 1e7, 1e7
# Train
for epoch in range(EPOCH):
  if debug:
    if epoch > 10:
      break
  model.train()
  train_mae, train_rmse, train_loss = train()
  model.eval()
  val_mae, val_rmse = val()
  # scheduler.step()
  stats.append((train_loss, train_mae, train_rmse, val_mae, val_rmse))
  stats_file = os.path.join(Log_Save_dir, "Log.txt")
  with open(stats_file, 'w') as f:
      for s in stats:
          f.write("%s\n" % ','.join([str(x) for x in s]))
  if best_mae >= val_mae:
    best_mae = val_mae
    best_rmse = val_rmse
    model_save_path = os.path.join(Log_Save_dir, "Best_Model.pth")
    torch.save(model.state_dict(), model_save_path)
  if (epoch + 1) % 5 == 0:
    print("\033[1;31;47mEpoch {}, Avg. Epoch Loss: {} Train MAE: {} Train RMSE: {} Val MAE: {} Val RMSE: {} Best Val MAE: {} Best Val RMSE: {} \033[0m".format(
              epoch + 1, stats[-1][0], stats[-1][1], stats[-1][2], stats[-1][3], stats[-1][4], best_mae, best_rmse))

# Test
model.load_state_dict(torch.load(model_save_path))
model.eval()
test_mae, test_rmse = test()
print("\033[1;33;44mTest MAE {}, Test RMSE {} \033[0m".format(test_mae, test_rmse))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


TRAIN MAE:  52.39, TRAIN RMSE:  95.30, Train Loss: 9.082234571350766, Running Time:0:10:21.687947
VAL MAE:  56.56, VAL RMSE: 126.17, Running Time: 0:01:59.529503
TRAIN MAE:  42.71, TRAIN RMSE:  86.50, Train Loss: 7.4819333498695855, Running Time:0:10:22.972392
VAL MAE:  55.27, VAL RMSE: 126.08, Running Time: 0:02:00.232673
TRAIN MAE:  41.73, TRAIN RMSE:  86.47, Train Loss: 7.47726194599113, Running Time:0:10:22.066357
VAL MAE:  54.30, VAL RMSE: 125.62, Running Time: 0:01:56.375376
TRAIN MAE:  41.41, TRAIN RMSE:  86.46, Train Loss: 7.474551947631009, Running Time:0:10:22.871654
VAL MAE:  54.15, VAL RMSE: 125.89, Running Time: 0:01:55.790475
TRAIN MAE:  41.65, TRAIN RMSE:  86.69, Train Loss: 7.5145979844955235, Running Time:0:10:22.300849
VAL MAE:  53.93, VAL RMSE: 125.77, Running Time: 0:01:58.535659
Epoch 5, Avg. Epoch Loss: 7.5145979844955235 Train MAE: 41.64828864262058 Train RMSE: 86.68674933224845 Val MAE: 53.92530010054308 Val RMSE: 125.77100789064626 Best Val MAE: 53.925300100543

In [ ]:
#We can see that the train loss at the end is the least of all, however least MAE I get is 40.87 in epoch 15. 
#If we ran it for more epochs we wcan probably get a better MAE, maybe even <10 if epochs~150


# Model training converged to local optima - 49.96
The logs and model weights can be found in the project folder under folder "run 1 - local optima"

In [ ]:
from lib.utils import TransformTrain
import gc

def train():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in train_loader:
      optimizer.zero_grad()
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        train_sample = train_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = train_sample['im_id'], train_sample['image'], train_sample['boxes'], train_sample['dots'], train_sample['bg_mask_img'], train_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        # Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)

          # reshape interpolate
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        # Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        # Feed to the Network
        # Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')

        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)

        #
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Loss 1
        loss = criterion(out_density, GT_density) + 1e-3 * (out_density.sum() - gt_cnt) ** 2

        #Loss 2
        #loss = criterion(out_density, GT_density)

        train_loss += loss.item()

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        loss.backward()
        endtime = datetime.datetime.now()
      optimizer.step()
    print('TRAIN MAE: {:6.2f}, TRAIN RMSE: {:6.2f}, Train Loss: {}, Running Time:{}'.format(SAE/cnt, (SSE/cnt)**0.5, train_loss/cnt, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    TRAIN_LOSS = train_loss/cnt
    return MAE, RMSE, TRAIN_LOSS

def val():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in val_loader:
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        val_sample = val_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = val_sample['im_id'], val_sample['image'], val_sample['boxes'], val_sample['dots'], val_sample['bg_mask_img'], val_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        #Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        #Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        #Feed to the Network
        #Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')
        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        endtime = datetime.datetime.now()
    print('VAL MAE: {:6.2f}, VAL RMSE: {:6.2f}, Running Time: {}'.format(SAE/cnt, (SSE/cnt)**0.5, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    return MAE, RMSE

def test():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in test_loader:
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        test_sample = test_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = test_sample['im_id'], test_sample['image'], test_sample['boxes'], test_sample['dots'], test_sample['bg_mask_img'], test_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        #Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        #Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        #Feed to the Network
        #Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')
        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        endtime = datetime.datetime.now()
    print('TEST MAE: {:6.2f}, TEST RMSE: {:6.2f}, Running Time: {}'.format(SAE/cnt, (SSE/cnt)**0.5, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    return MAE, RMSE


EPOCH = 20
debug = False
stats = list()
Log_Save_dir = './'
best_mae, best_rmse = 1e7, 1e7
# Train
for epoch in range(EPOCH):
  if debug:
    if epoch > 5:
      break
  model.train()
  train_mae, train_rmse, train_loss = train()
  model.eval()
  val_mae, val_rmse = val()
  stats.append((train_loss, train_mae, train_rmse, val_mae, val_rmse))
  stats_file = os.path.join(Log_Save_dir, "Log.txt")
  with open(stats_file, 'w') as f:
      for s in stats:
          f.write("%s\n" % ','.join([str(x) for x in s]))
  if best_mae >= val_mae:
    best_mae = val_mae
    best_rmse = val_rmse
    model_save_path = os.path.join(Log_Save_dir, "Best_Model.pth")
    torch.save(model.state_dict(), model_save_path)
  if (epoch + 1) % 5 == 0:
    gc.collect()
    torch.cuda.empty_cache()
    print("\033[1;31;47mEpoch {}, Avg. Epoch Loss: {} Train MAE: {} Train RMSE: {} Val MAE: {} Val RMSE: {} Best Val MAE: {} Best Val RMSE: {} \033[0m".format(
              epoch + 1, stats[-1][0], stats[-1][1], stats[-1][2], stats[-1][3], stats[-1][4], best_mae, best_rmse))

# Test
model.load_state_dict(torch.load(model_save_path))
model.eval()
test_mae, test_rmse = test()
print("\033[1;33;44mTest MAE {}, Test RMSE {} \033[0m".format(test_mae, test_rmse))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


TRAIN MAE:  50.22, TRAIN RMSE:  98.88, Train Loss: 9.776723294124828, Running Time:0:28:52.068553
VAL MAE:  63.54, VAL RMSE: 138.91, Running Time: 0:10:05.355836
TRAIN MAE:  49.96, TRAIN RMSE:  98.60, Train Loss: 9.721585216975523, Running Time:0:11:31.375491
VAL MAE:  63.54, VAL RMSE: 138.91, Running Time: 0:02:11.894140
TRAIN MAE:  49.96, TRAIN RMSE:  98.60, Train Loss: 9.7215852200706, Running Time:0:11:32.870879
VAL MAE:  63.54, VAL RMSE: 138.91, Running Time: 0:02:11.691916
TRAIN MAE:  49.96, TRAIN RMSE:  98.60, Train Loss: 9.7215852200706, Running Time:0:11:32.843777
VAL MAE:  63.54, VAL RMSE: 138.91, Running Time: 0:02:10.579335
TRAIN MAE:  49.96, TRAIN RMSE:  98.60, Train Loss: 9.721584775943505, Running Time:0:11:39.401419
VAL MAE:  63.54, VAL RMSE: 138.91, Running Time: 0:02:12.068581
Epoch 5, Avg. Epoch Loss: 9.721584775943505 Train MAE: 49.962830634033764 Train RMSE: 98.59808015399362 Val MAE: 63.53888024883359 Val RMSE: 138.90855098426243 Best Val MAE: 63.53887522568952 Be

#Result Explanation

In [ ]:
"""
    PART IV: 20 points
    How is your results? Why do you think it doesn't perform better? 
    How would you improve this model's accuracy?

    Answer these questions use what you have learned from this course and 
    material provided above.

Answer:

In this model I first tried to use the pre-trained weights from the ViT model. However,training this on colab 
proved to be a big hassle since it took almost an hour to run on the entire dataset of train and test set. To 
deal with this issue I avoided the 12 layer encoder, instead not assigining pretrained weights for the model weights
and leaving them as is. Instead I reduced the number of layers to 4 and number of heads to 5 to reduce the training
time, since there are lesser weights to learn now(Because of this we can't use the pretrained model). I am also running
the training for at the most 20 epochs, since colab kept on crashing or reaching GPU limit and I had to sit beside it to 
not let it fail.

This however, made the MAE swing back and forth and in most runs, converge to local minima of 49.96 MAE. To avoid 
this I tried several things:

1) Tried modifying the learning rate - this resulted in varying levels of convergence but most of the time it still
converged to local minima.

2) Tried adding decay using scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) this had no effect as well, 
   converged to local minima mostly/swung a lot.

3) I initialized the weights of both the Linear layers(fully connected) in the MLP block to some random weights using 
   torch.nn.init.xavier_uniform_(self.fc2.weight) method on the both fc layers in every encoder. This made the MAE start 
   at very high values and since our L.R was very low, it did not reduce much. So I removed one of the random xavier
   weight initializations and kept the one for fc2 layer only. This seemed to make the value converge with a L.R of 5e-5

4) Another approach I tried was to load the pretrained weights and load it to a 12 layer 12 head CountingTransformer. Then, 
   to speed up the process I removed the encoder layers from 4 to 12 from the initialized transformer and only kept the 
   first 4 encoder layers. This training was still quite fast but converged to loal minima in the 1st epoch itself.

I need to test this model more to decide how to optimize it further. We need more GPU resources to test this out. Given enough 
GPU resources, we can plot the variation of different hypermparamters with the best MAE achieved and then select the optimal 
hyperparamteres.

Our main issue over here is that we are limited by our GPU resources and duration of use. We cannot run the code more than 8 hours 
on colab. After this it crashes and you cannot access colab GPUs from the same account. Given enough GPUs we should train this model 
for different combination of hyperparameters discussed above for 100 epochs atleast since 20 is very less to determine anything in 
such a large network. We should then also be taking the pretrained weights from ViT and 12 layers for counting transformer and 12 heads 
for the multi-head attention.
"""

In [ ]:

"""

    BONUS: 30 points
    Modify this network according to your explanation, run it again.
    How does the loss value change compared with original model?

    When you do this port, save code and model in another file named
    CV_2021FALL_hw5.bonus.ipynb
    and attach a graph of your model, explantion input and output of parts clearly.

    Not sure. Not enough time.
"""
